In [2]:
import pandas as pd
import numpy as np
import sklearn as sk
import matplotlib.pyplot as plt
import datetime
from sklearn.cross_validation import train_test_split

In [3]:
DATA = pd.concat(pd.read_csv('dis_acct_e.csv', chunksize=1000, date_parser=pd.datetools.to_datetime))
TARGETData = 'bal_outst_amt'

In [4]:
DATA.head()

,me_dt,acct_id,acct_sub_type,acct_opnd_dt,acct_opng_brnch_id,acct_sts,cust_id,prod_id,fincg_type_cd,tenor,...,inlien_ind,cury_cd,bal_outst_amt,avg_bal_out_globl_amt,avg_bal_out_globl_amt_cr,avg_bal_out_globl_amt_dr,fund_cr_amt,fund_chrg_amt,inlien_amt,no_of_time_renewed
0,2015-12-31,18124905,CURRENT ACCOUNT/OVERDRAFT,2013-07-18,3426827,CLOSE,11716809,6302,191,NaN,...,N,MYR,16.289508,15.649508,15.969508,24.839508,15.679508,7.129508,16.259508,0
1,2015-12-31,17816466,CURRENT ACCOUNT/OVERDRAFT,2013-04-10,3426532,CLOSE,11503652,6302,191,NaN,...,N,MYR,73659.291778,44321.011778,58990.151778,78314.921778,44385.081778,39729.451778,73595.221778,0
2,2015-12-31,2189400,CURRENT ACCOUNT/OVERDRAFT,2013-11-27,3426268,ACTIVE,116097,5831,111,NaN,...,N,MYR,172067.371356,84835.531356,128451.451356,158898.531356,84934.011356,98102.851356,171968.891356,0
3,2015-12-31,17851353,CURRENT ACCOUNT/OVERDRAFT,2014-07-20,3426565,ACTIVE,12568229,6302,191,NaN,...,N,MYR,1969.166978,1094.726978,1531.946978,1977.716978,1096.206978,1087.656978,1967.686978,0
4,2015-12-31,18208198,CURRENT ACCOUNT/OVERDRAFT,2014-01-22,3426892,DORMANT,12123857,6302,191,NaN,...,N,MYR,44622.457314,24656.457314,34639.457314,44622.457314,24689.557314,24689.557314,44589.357314,0


In [52]:
print(DATA.columns)

Index([u'me_dt', u'acct_id', u'acct_sub_type', u'acct_opnd_dt',
       u'acct_opng_brnch_id', u'acct_sts', u'cust_id', u'prod_id',
       u'fincg_type_cd', u'tenor', u'tenor_uom', u'dep_maturity_dt',
       u'cert_iss_dt', u'remain_term', u'remain_term_uom',
       u'maturity_handle_codes', u'joint_acct_ind', u'new_acct_ind',
       u'inlien_ind', u'cury_cd', u'bal_outst_amt', u'avg_bal_out_globl_amt',
       u'avg_bal_out_globl_amt_cr', u'avg_bal_out_globl_amt_dr',
       u'fund_cr_amt', u'fund_chrg_amt', u'inlien_amt', u'no_of_time_renewed'],
      dtype='object')


In [57]:
DATA['remain_term_uom'].value_counts

<bound method Series.value_counts of 0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
5      NaN
6      NaN
7      NaN
8      NaN
9      NaN
10     NaN
11     NaN
12     NaN
13     NaN
14     NaN
15     NaN
16     NaN
17     NaN
18     NaN
19     NaN
20     NaN
21     NaN
22     NaN
23     NaN
24     NaN
25     NaN
26     NaN
27     NaN
28     NaN
29     NaN
      ... 
364    NaN
365    NaN
366    NaN
367    NaN
368    NaN
369    NaN
370    NaN
371    NaN
372    NaN
373    NaN
374    NaN
375    NaN
376    NaN
377    NaN
378    NaN
379    NaN
380    NaN
381    NaN
382    NaN
383    NaN
384    NaN
385    NaN
386    NaN
387    NaN
388    NaN
389    NaN
390    NaN
391    NaN
392    NaN
393    NaN
Name: remain_term_uom, dtype: object>

In [6]:
print(isinstance(DATA['acct_opnd_dt'].iloc[1], datetime.date))
print(DATA['acct_opnd_dt'].iloc[1])

False
2013-04-10


In [7]:
dateColumns = ['me_dt', 'acct_opnd_dt', 'cert_iss_dt', 'dep_maturity_dt']
for x in dateColumns:
    DATA[x] = pd.to_datetime(DATA[x])
    print(x, "is", DATA['me_dt'].dtype)

('me_dt', 'is', dtype('<M8[ns]'))
('acct_opnd_dt', 'is', dtype('<M8[ns]'))
('cert_iss_dt', 'is', dtype('<M8[ns]'))
('dep_maturity_dt', 'is', dtype('<M8[ns]'))


In [8]:
DATA_clean = DATA.set_index('acct_id').select_dtypes(exclude=['datetime64[ns]', np.flexible], include=[np.number])

In [9]:
DATA_clean.head()

,acct_opng_brnch_id,cust_id,prod_id,fincg_type_cd,tenor,remain_term,bal_outst_amt,avg_bal_out_globl_amt,avg_bal_out_globl_amt_cr,avg_bal_out_globl_amt_dr,fund_cr_amt,fund_chrg_amt,inlien_amt,no_of_time_renewed
acct_id,,,,,,,,,,,,,,
18124905,3426827,11716809,6302,191,NaN,0,16.289508,15.649508,15.969508,24.839508,15.679508,7.129508,16.259508,0
17816466,3426532,11503652,6302,191,NaN,0,73659.291778,44321.011778,58990.151778,78314.921778,44385.081778,39729.451778,73595.221778,0
2189400,3426268,116097,5831,111,NaN,0,172067.371356,84835.531356,128451.451356,158898.531356,84934.011356,98102.851356,171968.891356,0
17851353,3426565,12568229,6302,191,NaN,0,1969.166978,1094.726978,1531.946978,1977.716978,1096.206978,1087.656978,1967.686978,0
18208198,3426892,12123857,6302,191,NaN,0,44622.457314,24656.457314,34639.457314,44622.457314,24689.557314,24689.557314,44589.357314,0


In [10]:
def cleanNulls(data):
    for x in data:
        sumNull = data[x].isnull().sum()
        if len(data) / sumNull > 0.5:
            print("dropping ", x)
            data = data.drop(x, axis=1)
    return data

DATA_clean = cleanNulls(DATA_clean)

('dropping ', 'tenor')


In [50]:
def checkNulls(data):
    for x in data:
        print(x)
        print(data[x].isnull().sum(), len(data) / data[x].isnull().sum())

checkNulls(DATA_clean)

acct_opng_brnch_id
(0, 0)
cust_id
(0, 0)
prod_id
(0, 0)
fincg_type_cd
(0, 0)
remain_term
(0, 0)
bal_outst_amt
(0, 0)
avg_bal_out_globl_amt
(0, 0)
avg_bal_out_globl_amt_cr
(0, 0)
avg_bal_out_globl_amt_dr
(0, 0)
fund_cr_amt
(0, 0)
fund_chrg_amt
(0, 0)
inlien_amt
(0, 0)
no_of_time_renewed
(0, 0)


In [12]:
DATA_noInt = DATA.set_index('acct_id').select_dtypes(exclude=['floating', 'int', 'datetime64[ns]'])
DATA_noInt.head()

,acct_sub_type,acct_sts,tenor_uom,remain_term_uom,maturity_handle_codes,joint_acct_ind,new_acct_ind,inlien_ind,cury_cd
acct_id,,,,,,,,,
18124905,CURRENT ACCOUNT/OVERDRAFT,CLOSE,NaN,NaN,NaN,N,N,N,MYR
17816466,CURRENT ACCOUNT/OVERDRAFT,CLOSE,NaN,NaN,NaN,N,N,N,MYR
2189400,CURRENT ACCOUNT/OVERDRAFT,ACTIVE,NaN,NaN,NaN,N,N,N,MYR
17851353,CURRENT ACCOUNT/OVERDRAFT,ACTIVE,NaN,NaN,NaN,N,N,N,MYR
18208198,CURRENT ACCOUNT/OVERDRAFT,DORMANT,NaN,NaN,NaN,N,N,N,MYR


In [51]:
checkNulls(DATA_noInt)

acct_sub_type
(0, 0)
acct_sts
(0, 0)
tenor_uom
(1006236, 1)
remain_term_uom
(1006236, 1)
maturity_handle_codes
(1084324, 1)
joint_acct_ind
(0, 0)
new_acct_ind
(0, 0)
inlien_ind
(0, 0)
cury_cd
(0, 0)


In [13]:
DATA_noInt.iloc[:,0:].apply(pd.Series.value_counts)

,acct_sub_type,acct_sts,tenor_uom,remain_term_uom,maturity_handle_codes,joint_acct_ind,new_acct_ind,inlien_ind,cury_cd
ACTIVE,NaN,1008399.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AUTO RENEW,NaN,NaN,NaN,NaN,59541.0,NaN,NaN,NaN,NaN
CLOSE,NaN,100953.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CURRENT ACCOUNT/OVERDRAFT,1006236.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DORMANT,NaN,35042.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
FIXED DEPOSIT - CERTIFICATE,138158.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
MONTHS,NaN,NaN,138158.0,138158.0,NaN,NaN,NaN,NaN,NaN
MYR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1144394.0
N,NaN,NaN,NaN,NaN,NaN,1144274.0,1128424.0,1126376.0,NaN
NO AUTO RENEW,NaN,NaN,NaN,NaN,529.0,NaN,NaN,NaN,NaN


In [14]:
DATA_noInt_Vec = pd.get_dummies(DATA_noInt).astype('float32')

In [15]:
print(len(DATA_noInt_Vec))
DATA_noInt_Vec.head()

1144394


,acct_sub_type_CURRENT ACCOUNT/OVERDRAFT,acct_sub_type_FIXED DEPOSIT - CERTIFICATE,acct_sts_ACTIVE,acct_sts_CLOSE,acct_sts_DORMANT,tenor_uom_MONTHS,remain_term_uom_MONTHS,maturity_handle_codes_AUTO RENEW,maturity_handle_codes_NO AUTO RENEW,joint_acct_ind_N,joint_acct_ind_Y,new_acct_ind_N,new_acct_ind_Y,inlien_ind_N,inlien_ind_Y,cury_cd_MYR
acct_id,,,,,,,,,,,,,,,,
18124905,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
17816466,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
2189400,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
17851353,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
18208198,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0


In [16]:
# print(dis_acct_e.columns.difference(dis_acct_e_nonInt.columns))
# dis_acct_e_int1 = dis_acct_e.ix[:, dis_acct_e.columns.difference(dis_acct_e_nonInt.columns)]
# dis_acct_e_int1.head()

In [17]:
# dis_acct_e_int2 = dis_acct_e_int1.set_index('acct_id')
# dis_acct_e_nonInt_vec.head()
# pd.merge(dis_acct_e_int2, dis_acct_e_nonInt_vec, on=dis_acct_e_int2.index)

In [18]:
DATA_clean_Vec = DATA_clean.join(DATA_noInt_Vec)

In [19]:
DATA_clean_Vec.columns

Index([u'acct_opng_brnch_id', u'cust_id', u'prod_id', u'fincg_type_cd',
       u'remain_term', u'bal_outst_amt', u'avg_bal_out_globl_amt',
       u'avg_bal_out_globl_amt_cr', u'avg_bal_out_globl_amt_dr',
       u'fund_cr_amt', u'fund_chrg_amt', u'inlien_amt', u'no_of_time_renewed',
       u'acct_sub_type_CURRENT ACCOUNT/OVERDRAFT',
       u'acct_sub_type_FIXED DEPOSIT - CERTIFICATE', u'acct_sts_ACTIVE',
       u'acct_sts_CLOSE', u'acct_sts_DORMANT', u'tenor_uom_MONTHS',
       u'remain_term_uom_MONTHS', u'maturity_handle_codes_AUTO RENEW',
       u'maturity_handle_codes_NO AUTO RENEW', u'joint_acct_ind_N',
       u'joint_acct_ind_Y', u'new_acct_ind_N', u'new_acct_ind_Y',
       u'inlien_ind_N', u'inlien_ind_Y', u'cury_cd_MYR'],
      dtype='object')

In [20]:
DATA_clean_Vec.head(900000)

,acct_opng_brnch_id,cust_id,prod_id,fincg_type_cd,remain_term,bal_outst_amt,avg_bal_out_globl_amt,avg_bal_out_globl_amt_cr,avg_bal_out_globl_amt_dr,fund_cr_amt,...,remain_term_uom_MONTHS,maturity_handle_codes_AUTO RENEW,maturity_handle_codes_NO AUTO RENEW,joint_acct_ind_N,joint_acct_ind_Y,new_acct_ind_N,new_acct_ind_Y,inlien_ind_N,inlien_ind_Y,cury_cd_MYR
acct_id,,,,,,,,,,,,,,,,,,,,,
938277,3426102,11746706,5831,111,0,1.910189e+01,17.201892,1.815189e+01,2.765189e+01,17.231892,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
938277,3426102,11746706,5831,111,0,1.910189e+01,17.201892,1.815189e+01,2.765189e+01,17.231892,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
938277,3426102,11746706,5831,111,0,1.910189e+01,17.201892,1.815189e+01,2.765189e+01,17.231892,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
938277,3426102,11746706,5831,111,0,1.910189e+01,17.201892,1.815189e+01,2.765189e+01,17.231892,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
938277,3426102,11746706,5831,111,0,1.910189e+01,17.201892,1.815189e+01,2.765189e+01,17.231892,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
938277,3426102,11746706,5831,111,0,1.910189e+01,17.201892,1.815189e+01,2.765189e+01,17.231892,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
938277,3426102,11746706,5831,111,0,4.240897e+00,2.340897,3.290897e+00,4.240897e+00,2.340897,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
938277,3426102,11746706,5831,111,0,4.240897e+00,2.340897,3.290897e+00,4.240897e+00,2.340897,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
938277,3426102,11746706,5831,111,0,4.240897e+00,2.340897,3.290897e+00,4.240897e+00,2.340897,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0


In [34]:
y = DATA_clean_Vec.pop(TARGETData)
X = DATA_clean_Vec.head(1000000)

In [36]:
X_train,X_test,y_train,y_test = train_test_split(X,y.head(1000000),test_size=0.2)

In [37]:
# iX.loc[X_train] # return dataframe train
print(len(X_train))
X_train

800000


,acct_opng_brnch_id,cust_id,prod_id,fincg_type_cd,remain_term,avg_bal_out_globl_amt,avg_bal_out_globl_amt_cr,avg_bal_out_globl_amt_dr,fund_cr_amt,fund_chrg_amt,...,remain_term_uom_MONTHS,maturity_handle_codes_AUTO RENEW,maturity_handle_codes_NO AUTO RENEW,joint_acct_ind_N,joint_acct_ind_Y,new_acct_ind_N,new_acct_ind_Y,inlien_ind_N,inlien_ind_Y,cury_cd_MYR
acct_id,,,,,,,,,,,,,,,,,,,,,
17353994,3426315,13347931,5831,111,0,2.039735e+05,3.494705e+05,4.055589e+05,2.041549e+05,2.935634e+05,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
2999420,3426624,12865241,6302,191,0,1.171448e+01,1.645448e+01,2.119448e+01,1.173448e+01,1.173448e+01,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
9353044,3426501,12147267,6302,191,0,1.138498e+05,1.459435e+05,1.979466e+05,1.140135e+05,9.410415e+04,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
2208826,3426464,116763,5831,111,0,1.367367e+05,2.012776e+05,2.527824e+05,1.368985e+05,1.499345e+05,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
17165189,3426188,13628238,5831,111,0,4.241448e+02,5.958948e+02,7.676448e+02,4.246848e+02,4.246848e+02,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
9367686,3426710,11770231,6302,191,0,7.439357e+01,1.041136e+02,1.344036e+02,7.449357e+01,7.392357e+01,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
17205348,3426219,13646040,5831,111,0,1.836171e+04,2.465602e+04,3.257175e+04,1.838683e+04,1.676541e+04,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
2198542,3426372,117359,5831,111,0,3.496724e+04,5.160250e+04,6.471754e+04,3.500966e+04,3.852988e+04,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
9375552,3426805,12597293,6302,191,0,7.946973e+02,1.116497e+03,1.438297e+03,7.957073e+02,7.957073e+02,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0


In [30]:
X.index

Int64Index([  938277,   938277,   938277,   938277,   938277,   938277,
              938277,   938277,   938277,   938277,
            ...
            88829280, 88829280, 88829380, 88829380, 88829380, 88829380,
            88829464, 88829464, 88829464, 88829464],
           dtype='int64', name=u'acct_id', length=9233782)

In [31]:
X.head(100)

,acct_opng_brnch_id,cust_id,prod_id,fincg_type_cd,remain_term,avg_bal_out_globl_amt,avg_bal_out_globl_amt_cr,avg_bal_out_globl_amt_dr,fund_cr_amt,fund_chrg_amt,...,remain_term_uom_MONTHS,maturity_handle_codes_AUTO RENEW,maturity_handle_codes_NO AUTO RENEW,joint_acct_ind_N,joint_acct_ind_Y,new_acct_ind_N,new_acct_ind_Y,inlien_ind_N,inlien_ind_Y,cury_cd_MYR
acct_id,,,,,,,,,,,,,,,,,,,,,
938277,3426102,11746706,5831,111,0,17.201892,18.151892,27.651892,17.231892,8.681892,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
938277,3426102,11746706,5831,111,0,17.201892,18.151892,27.651892,17.231892,8.681892,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
938277,3426102,11746706,5831,111,0,17.201892,18.151892,27.651892,17.231892,8.681892,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
938277,3426102,11746706,5831,111,0,17.201892,18.151892,27.651892,17.231892,8.681892,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
938277,3426102,11746706,5831,111,0,17.201892,18.151892,27.651892,17.231892,8.681892,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
938277,3426102,11746706,5831,111,0,17.201892,18.151892,27.651892,17.231892,8.681892,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
938277,3426102,11746706,5831,111,0,2.340897,3.290897,4.240897,2.340897,2.340897,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
938277,3426102,11746706,5831,111,0,2.340897,3.290897,4.240897,2.340897,2.340897,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
938277,3426102,11746706,5831,111,0,2.340897,3.290897,4.240897,2.340897,2.340897,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0


In [41]:
print(DATA_clean_Vec['remain_term'].value_counts)
print(len(DATA_clean_Vec['remain_term']))

<bound method Series.value_counts of acct_id
938277      0
938277      0
938277      0
938277      0
938277      0
938277      0
938277      0
938277      0
938277      0
938277      0
938277      0
938277      0
938277      0
938277      0
938277      0
938277      0
938277      0
938277      0
938277      0
938277      0
938277      0
938277      0
938277      0
938277      0
938277      0
938277      0
938277      0
938277      0
938277      0
938277      0
           ..
88828867    5
88828867    5
88828868    5
88828868    5
88828868    5
88828868    5
88828909    5
88828909    5
88828909    5
88828909    5
88829181    5
88829181    5
88829181    5
88829181    5
88829274    9
88829274    9
88829274    9
88829274    9
88829280    1
88829280    1
88829280    1
88829280    1
88829380    1
88829380    1
88829380    1
88829380    1
88829464    6
88829464    6
88829464    6
88829464    6
Name: remain_term, dtype: int64>
9233782
